In [2]:
from forecasting_functions import *

# importing the data
Stijn = True
fullYearpath = None
av_temperaturepath = None

if Stijn:
    fullYearpath = "D:\Onedrive\Leuven\Final project\data\Forecasting_writtendata\FullYear.csv"
    av_temperaturepath = "D:\Onedrive\Leuven\Final project\data\weather-avg.csv"
else:
    raise Exception("Put some paths here.")
fullYeardata = pd.read_csv(fullYearpath,index_col= "date",parse_dates= True)
av_temperature = pd.read_csv(av_temperaturepath,index_col='meter_id')
av_temperature = av_temperature.transpose()
av_temperature.index = pd.to_datetime(av_temperature.index)



name = fullYeardata.columns[1]
TS = fullYeardata[name]
temperature = av_temperature[name]



# all the training dates are present, dates from the test set can be missing.

# normalize the data --> min/max method (when using a single time serie)
temperature_norm,scaler_temperature = norm_forcast(temperature)
temperature_norm = substitute_missing_values_temperature(temperature_norm)
TS_norm,scaler_history = norm_forcast(TS)
temp = TS_norm[TS_norm.index.month == 1]
# temp = TS_norm[TS_norm.index.month != 12]
# test = TS_norm[TS_norm.index.month == 12]

# remove from the test set all the days that contain nan values -> only estimate real days
# test.dropna(inplace=True)

# substitute the missing values
temp = substitute_missing_values(temp)
training = temp[0:336]
validation = temp[336:384]
test = temp[384:528]
# training = temp[temp.index.month != 11]
# validation = temp[temp.index.month == 11]
TS_norm_full = substitute_missing_values(TS_norm.copy(deep=True))

amount of missing values: 0. 

amount of missing values: 0. 

amount of missing values: 0. 



In [3]:
X,y = input_output_LSTM(training, temperature_norm, 3)
X_val, y_val = input_output_LSTM(validation, temperature_norm, 3)
X.shape


(333, 3, 59)

In [4]:
from keras.layers import Dense,  LSTM,Embedding
from keras.models import Sequential, save_model, load_model, Input, Model
from keras import regularizers
from keras.callbacks import EarlyStopping,ModelCheckpoint, History
inputs1 = Input(shape=(X.shape[1], X.shape[2]))
lstm1, state_h, state_c = LSTM(1, return_state=True, return_sequences=True)(inputs1)

model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])
print(model.predict(np.zeros((1,3,59))))

Using TensorFlow backend.



[array([[[0.],
        [0.],
        [0.]]], dtype=float32), array([[0.]], dtype=float32), array([[0.]], dtype=float32)]


In [5]:

from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array
# define model
inputs1 = Input(batch_shape=(1,3, 1))
lstm1, state_h, state_c = LSTM(1, stateful= True, return_state= True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])
# define input data
data = array([[0,0,0], [0.1, 0.2, 0.3],[0,0,0],[0,0,0],[0.6,0.3,0.6]])
# make and show prediction
for row in data:
    row = row.reshape(1,3,1)
    print(model.predict(row))
    # model.reset_states()
    print(10*'-')

[array([[0.]], dtype=float32), array([[0.]], dtype=float32), array([[0.]], dtype=float32)]
----------
[array([[-0.0705552]], dtype=float32), array([[-0.0705552]], dtype=float32), array([[-0.13911606]], dtype=float32)]
----------
[array([[-0.03891772]], dtype=float32), array([[-0.03891772]], dtype=float32), array([[-0.07932621]], dtype=float32)]
----------
[array([[-0.02223484]], dtype=float32), array([[-0.02223484]], dtype=float32), array([[-0.04493025]], dtype=float32)]
----------
[array([[-0.15642524]], dtype=float32), array([[-0.15642524]], dtype=float32), array([[-0.31330794]], dtype=float32)]
----------


In [6]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array
# define model
inputs1 = Input(shape=(3, 1))
lstm1, state_h, state_c = LSTM(1, return_state= True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])
# define input data
data = array([[0,0,0], [0.1, 0.2, 0.3],[0,0,0],[0,0,0],[0.6,0.3,0.6]])
data = data.reshape((5,3,1))
# make and show prediction
# for row in data:
#     row = row.reshape(1,3,1)
output = model.predict(data)
print(model.predict(data))
print(10*'-')


[array([[ 0.        ],
       [-0.04422202],
       [ 0.        ],
       [ 0.        ],
       [-0.0791264 ]], dtype=float32), array([[ 0.        ],
       [-0.04422202],
       [ 0.        ],
       [ 0.        ],
       [-0.0791264 ]], dtype=float32), array([[ 0.        ],
       [-0.09709829],
       [ 0.        ],
       [ 0.        ],
       [-0.1930851 ]], dtype=float32)]
----------


In [7]:

from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array

# define model
inputs1 = Input(shape=(3, 1))
lstm1, state_h, state_c = LSTM(1, return_state=True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])
# define input data
data = array([0.1, 0.2, 0.3]).reshape((1,3,1))
# make and show prediction
print(model.predict(data))


[array([[0.00793442]], dtype=float32), array([[0.00793442]], dtype=float32), array([[0.01880421]], dtype=float32)]


In [15]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array
# define model
inputs1 = Input(input_shape=(5,3, 1))
lstm1 = LSTM(1, stateful= False)(inputs1)
model = Model(inputs=inputs1, outputs=lstm1)
# define input data
data = array([[0,0,0], [0.1, 0.2, 0.3],[0,0,0],[0,0,0],[0.6,0.3,0.6]])
# make and show prediction
# for row in data:
#     row = row.reshape(5,3,1)
print(model.predict(data.reshape((5,3,1)), batch_size= 5))
# model.reset_states()
print(10*'-')

[[ 0.        ]
 [-0.02127535]
 [ 0.        ]
 [ 0.        ]
 [-0.0457112 ]]
----------


In [12]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array
data = array([[0,0,0], [0.1, 0.2, 0.3],[0,0,0],[0,0,0],[0.6,0.3,0.6]])
model = Sequential()
model.add(LSTM(1, input_shape=(3,1)))

# for row in data:
#     row = row.reshape(1,3,1)
#     print(model.predict(row))
#     # model.reset_states()
#     print(10*'-')
model.predict(data )


[[0.]]
----------
[[-0.07480135]]
----------
[[0.]]
----------
[[0.]]
----------
[[-0.15030909]]
----------


In [21]:
from pandas import DataFrame
from pandas import concat
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
# create sequence
length = 10
sequence = [i/float(length) for i in range(length)]
# create X/y pairs
df = DataFrame(sequence)
df = concat([df, df.shift(1)], axis=1)
df.dropna(inplace=True)
# convert to LSTM friendly format
values = df.values
X, y = values[:, 0], values[:, 1]
X = X.reshape(len(X), 1, 1)
# configure network
n_batch = len(X)
n_epoch = 1000
n_neurons = 10
# design network
model = Sequential()
model.add(LSTM(n_neurons, input_shape=(X.shape[1], X.shape[2]), stateful=False))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
# fit network
for i in range(n_epoch):
	model.fit(X, y, epochs=1, batch_size=n_batch, verbose=0, shuffle=False)
	model.reset_states()
# online forecast
for i in range(len(X)):
	testX, testy = X[i], y[i]
	testX = testX.reshape(1, 1, 1)
	yhat = model.predict(testX, batch_size=1)
	print('>Expected=%.1f, Predicted=%.1f' % (testy, yhat))

>Expected=0.0, Predicted=0.0
>Expected=0.1, Predicted=0.1
>Expected=0.2, Predicted=0.2
>Expected=0.3, Predicted=0.3
>Expected=0.4, Predicted=0.4
>Expected=0.5, Predicted=0.5
>Expected=0.6, Predicted=0.6
>Expected=0.7, Predicted=0.7
>Expected=0.8, Predicted=0.8
